# Extract and align data from Onix, Harp, Sleap, and photometry

In [ ]:
import numpy as np
from pathlib import Path
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd
import harp

from harp_resources import process, utils
from sleap import load_and_process as lp

In [ ]:
cohort0 = False #only read harp data when it exists, not in Cohort0 
cohort2 = False

#Cohort 1 single OnixDigital file
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/VestibularMismatch_day1/B6J2717-2024-12-12T13-00-21') #single onix_digital files

#Cohort 1 multiple OnixDigital files 
data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/VestibularMismatch_day1/B6J2718-2024-12-12T13-28-14') #multiple onix_digital file

#Cohort 1 with clock accumulation issue marked on google sheet 
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/VestibularMismatch_day1/B6J2719-2024-12-12T13-59-38') #multiple onix_digital file

#Cohort 0 (no OnixHarp in this Cohort)
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort0/Cohort0_GCaMP_example/B3M3xx-2024-08-08T10-05-26')
#cohort0 = True

#Cohort 2 N.B. no videodata or photometry in this test set 
# cohort2 = True
# data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort2_like_test_data/2025-01-13T15-47-26')

#Cohort 2 N.B. no photometry in this test set (neitjer videos, but yes video_data)
# cohort2 = True
# data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort2_test_longer/2025-02-07T11-50-14')

photometry_path = data_path.parent / f"{data_path.name}_processedData" / "photometry"

# h1_datafolder = data_path / 'HarpDataH1' #only if reading separate registers
# h2_datafolder = data_path / 'HarpDataH2' #only if reading separate registers

In [ ]:
#h1 and h2 only needed if timestamps are readed separately and not as all harp_streams
# h1_reader = harp.create_reader('harp_resources/h1-device.yml', epoch=harp.REFERENCE_EPOCH)
# h2_reader = harp.create_reader('harp_resources/h2-device.yml', epoch=harp.REFERENCE_EPOCH)

session_settings_reader = utils.SessionData("SessionSettings")
experiment_events_reader = utils.TimestampedCsvReader("ExperimentEvents", columns=["Event"])
onix_framecount_reader = utils.TimestampedCsvReader("OnixAnalogFrameCount", columns=["Index"])
photometry_reader = utils.PhotometryReader("Processed_fluorescence")
video_reader1 = utils.Video("VideoData1")
video_reader2 = utils.Video("VideoData2")
onix_digital_reader = utils.OnixDigitalReader("OnixDigital", columns=["Value.Clock", "Value.HubClock", 
                                                                         "Value.DigitalInputs",
                                                                         "Seconds"])
onix_harp_reader = utils.TimestampedCsvReader("OnixHarp", columns=["Clock", "HubClock", "HarpTime"])

In [ ]:
#read metadata in 2 different ways (to df or to dict, to decide which one is better in the future)
print ("Loading session settings")
session_settings = utils.load_2(session_settings_reader, data_path) #Andrew's, creates ugly df, but used in further analysis code
#session_settings = utils.read_SessionSettings(data_path) #Hilde's, creates prety dict, not aware of multiple files

# read experiment events, video, processed photometry 
print ("Loading experiment events")
experiment_events = utils.load_2(experiment_events_reader, data_path)
if not cohort2:
    print ("Loading processed fluorescence")
    photometry_data=pd.read_csv(str(photometry_path)+'/Processed_fluorescence.csv')
    print ("Loading processed fluorescence info")
    photometry_info=pd.read_csv(str(photometry_path)+'/Info.csv')
    print ("Loading processed fluorescence events")
    photometry_events=pd.read_csv(str(photometry_path)+'/Events.csv')
    print ("Loading video data 1")
    video_data1 = utils.load_2(video_reader1, data_path)
    print ("Loading video data 2")
    video_data2 = utils.load_2(video_reader2, data_path)

# read Onix data 
print ("Loading OnixDigital")
onix_digital = utils.load_2(onix_digital_reader, data_path)
if not cohort0:
    print ("Loading OnixHarp")
    onix_harp = utils.load_2(onix_harp_reader, data_path)
    # removing possible outliers 
    onix_harp = utils.detect_and_remove_outliers(
    df=onix_harp,
    x_column="HarpTime",
    y_column="Clock",
    verbose=False  # True prints all outliers
    )
    
print ("Loading OnixAnalogFrameClock")
onix_analog_framecount = utils.load_2(onix_framecount_reader, data_path)
print ("Loading OnixAnalogClock")
onix_analog_clock = utils.read_OnixAnalogClock(data_path)
print ("Loading OnixAnalogData")
onix_analog_data = utils.read_OnixAnalogData(data_path, channels = [0], binarise=True) #channels is a list of AI lines, 0-11

#read harp streams and separate registers if needed 
print ("Loading H1 and H2 streams as dict or df")
harp_streams = utils.load_registers(data_path, dataframe = True) #loads as df, or if False, as dict 

# print (" ")
# print ("loading separate registers from H1 and H2 data")
# print ("Loading camera triggers")
# camera_triggers = utils.load_harp(h1_reader.Cam0Event, h1_datafolder) #assumes Cam0 triggers both cameras
# print ("Loading flow sensor data")
# flow_sensor = utils.load_harp(h1_reader.OpticalTrackingRead, h1_datafolder)
print ("Done Loading")

In [ ]:
%whos

### DEV align Onix, HARP and Photometry data 

In [ ]:
import importlib
importlib.reload(utils)  # Forces Python to reload the updated module
None

In [ ]:
conversions = process.photometry_alingment_Cohort1plus(
    onix_analog_clock, 
    onix_analog_framecount,
    onix_digital,
    photometry_events, 
    verbose=True
)


### iterate through folders to load OnixHarp and test outliers. 
### beginning can be used to iterate through folder for batch processing 

In [ ]:
# Define root directory
root_dir = Path("/Users/rancze/Documents/Data/vestVR/Cohort1/VestibularMismatch_day1/")

# Initialize dictionary for storing paths
data_paths = {}

# List directories one level down and add them to the dictionary
for sub_dir in root_dir.iterdir():  # Only iterate one level down
    if sub_dir.is_dir() and not sub_dir.name.endswith("_processedData") and not sub_dir.name.startswith("."):
        data_paths[sub_dir.name] = sub_dir  # Use the directory name as the key and the full path as the value

# Iterate over the dictionary and run the command
for key, data_path in data_paths.items():
    print(f"Loading: {key}")
    print " "
    onix_harp = utils.load_2(onix_harp_reader, data_path)
    # removing possible outliers 
    onix_harp = utils.detect_and_remove_outliers(
    df=onix_harp,
    x_column="HarpTime",
    y_column="Clock",
    verbose=False  # True prints all outliers
    )